In [21]:
import numpy as np
from joblib import Parallel, delayed
import random
import pickle

# Constants
budget = 10**10
alpha = 0.05
a = 0.45
b = 0.50


# Simulate a single test
def gen_test(p_test, eps, a, b, m, r):
    # Generate m query probabilities uniformly between A and B, and append p_new
    p_list = np.concatenate([np.random.uniform(a, b, size=m), [p_test]])

    # Estimate Y-probabilities using binomial sampling
    p_hat_list = np.array([np.mean(np.random.binomial(1, p, r)) for p in p_list])

    # Calculate the test statistic
    T_mr = np.min(np.abs(p_hat_list[:m] - p_hat_list[m]))

    return 1 if T_mr > eps else 0


# Estimate power of the test
def power_sim(p_test, eps, a,b, m, r, n_mc=50):
    results = [gen_test(p_test, eps, a, b, m, r) for _ in range(n_mc)]
    return np.mean(results)


# Load bounds (simulate loading "Bounds.RData")
def load_bounds(filename="Bounds.pkl"):
    with open(filename, "rb") as f:
        df_bounds = pickle.load(f)
    return df_bounds


# Empirical power estimate using joblib
def estimate_average_power(df_bounds, a, b):
    max_hhat_idx = np.argmax(df_bounds["hhat"])
    eps_star = df_bounds["thres"][max_hhat_idx]
    m_star = df_bounds["m"][max_hhat_idx]
    r_star = df_bounds["r"][max_hhat_idx]

    print("Optimal values:", eps_star, m_star, r_star)

    # Generate values from alternative region
    p_test_list = np.random.uniform(0, 1, 100)
    p_test_list = p_test_list[(p_test_list < a) | (p_test_list > b)]

    # Parallel power estimation with joblib
    BB = Parallel(n_jobs=-1)(
        delayed(power_sim)(p_test, eps_star, a, b, m_star, r_star)
        for p_test in p_test_list
    )

    print("Estimated average power:", np.mean(BB))
    return BB

In [22]:
# Example usage:
# Simulate loading a Bounds table (replace with actual loading logic)
df_bounds = {
    "thres": np.linspace(0.01, 0.03, 10),
    "m": np.linspace(100, 150, 10, dtype=int),
    "r": np.linspace(10e3, 10e4, 10, dtype=int),
    "hhat": np.random.rand(10)
}

with open("Bounds.pkl", "wb") as f:
    pickle.dump(df_bounds, f)

bounds = load_bounds("Bounds.pkl")
estimate_average_power(bounds, a, b)

Optimal values: 0.014444444444444444 111 30000
Estimated average power: 0.9773195876288661


[0.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9,
 1.0,
 1.0,
 1.0,
 0.78,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.96,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.9,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 0.26,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0,
 1.0]